# Converter from pommesinvest to AMIRIS
Convert pommesinvest results to AMIRIS input data format

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pommesevaluation.amiris_converter import (
    convert_annual_data_to_fame_time, convert_time_series_index_to_fame_time
)

## Notebook and workflow settings

In [ ]:
# Simulation settings
time_frame_in_years = 31
freq = "8H"
multiplier = {
    "1H": 1,
    "4H": 4,
    "8H": 8,
}

# Paths to data needed to be converted
path_model_inputs = "./model_inputs/"
path_model_results = "./model_results/"
path_processed_outputs = "./data_out/"

# Filename and scenarios
filename_investment = "results_total_dr_scenario"
filename_backup = "installed_capacity_backup_generation.csv"
filename_storage = "installed_storage_capacity.csv"
file_name_res_capacities = "sources_renewables_investment_model.csv"
file_name_res_generation = "sources_renewables_ts_hourly.csv"

dr_scenarios = ["none", "5", "50", "95"]

file_names_demand = {
    scen: f"sinks_demand_el_excl_demand_response_{scen}.csv"
    for scen in dr_scenarios
}
file_names_demand_ts = {
    scen: f"sinks_demand_el_excl_demand_response_ts_{scen}_hourly.csv"
    for scen in dr_scenarios
}
file_names_demand["none"] = "sinks_demand_el.csv"
file_names_demand_ts["none"] = "sinks_demand_el_ts_hourly.csv"

file_names_dispatch_results = {
    scen: (
        f"investment_LP_start-2020-01-01_{time_frame_in_years}"
        f"-years_simple_freq_{freq}_with_dr_{scen}_production.csv"
    ) for scen in dr_scenarios
}
file_names_dispatch_results["none"] = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{freq}_no_dr_50_production.csv"
)

# Define which pieces of information to exclude from investments
exclude_from_investment = [
    "PHS_inflow", 
    "battery_inflow", 
    "hoho_cluster_shift_only",
    "hoho_cluster_shift_shed",
    "hydrogen_electrolyzer",
    "ind_cluster_shed_only",
    "ind_cluster_shift_only",
    "ind_cluster_shift_shed",
    "tcs+hoho_cluster_shift_only",
    "tcs_cluster_shift_only",
]

demand_response_focus_cluster = "ind_cluster_shift_only"
exclude_from_investment.remove(demand_response_focus_cluster)

# Read in and convert
## Transformers & renewables
* Different for all scnearios: Investment results (transformers only)
* Same across all scenarios:
    * Backup generation (exogenous transformers)
    * Renewable capacities & generation

In [ ]:
# Investments per scenario
for dr_scenario in dr_scenarios:
    invest_results = pd.read_csv(f"{path_processed_outputs}{filename_investment}_{dr_scenario}.csv", index_col=0)
    invest_results = invest_results.loc[[idx for idx in invest_results.index if idx not in exclude_from_investment]]
    _ = convert_annual_data_to_fame_time(
        invest_results, 
        save=True, 
        path=f"./data_out/amiris/{dr_scenario}/",
        filename="installed_capacity_ts"
    )

In [ ]:
# Backup and RES
backup_capacities = pd.read_csv(f"{path_processed_outputs}{filename_backup}", index_col=0)

res_generation = pd.read_csv(f"{path_model_inputs}{file_name_res_generation}", index_col=0)

_ = convert_annual_data_to_fame_time(
    backup_capacities,
    save=True,
    path="./data_out/amiris/all_scenarios/",
    filename="exogenous_installed_capacity_ts"
)
_ = convert_time_series_index_to_fame_time(
    res_generation,
    save=True,
    path="./data_out/amiris/all_scenarios/",
    filename="res_generation_ts"
)

res_capacities_2020 = pd.read_csv(f"{path_model_inputs}{file_name_res_capacities}", index_col=0)
res_capacities_2020 = res_capacities_2020.loc[res_capacities_2020["country"] == "DE"]
res_capacities_2020.index = res_capacities_2020.index.str.split("_", expand=True).get_level_values(2)
res_capacities_2020 = res_capacities_2020["capacity"]
res_capacities_2020.to_csv("installed_renewable_capacities_2020.csv")

## "Net" Demand
Since in AMIRIS, there is a shortcoming in terms of modelling competing demand-side flexibility options, all flexibility options except the one demand response-cluster that is focused upon are not modelled explicitly, but their dispatch is considered in calculating a net demand, i.e. a 
* demand after
    * imports and exports,
    * storages,
    * demand response (except for the focus cluster)
    * electrolyzer operation

In [ ]:
original_demand = {}
demand_after_flex = {}

for dr_scenario in dr_scenarios:
    # Original demand
    sinks_demand_el = pd.read_csv(f"{path_model_inputs}{file_names_demand[dr_scenario]}", index_col=0)
    max_demand = sinks_demand_el.at["DE_sink_el_load", "maximum"]
    sinks_demand_el_ts = pd.read_csv(f"{path_model_inputs}{file_names_demand_ts[dr_scenario]}", index_col=0)
    original_demand[dr_scenario] = sinks_demand_el_ts["DE_sink_el_load"] * max_demand
    
    demand_after_flex = original_demand[dr_scenario].copy()
    demand_after_flex.index = pd.to_datetime(demand_after_flex.index)
    
    # Read in dispatch results
    dispatch_results = pd.read_csv(f"{path_model_results}{file_names_dispatch_results[dr_scenario]}", index_col=0)
    
    # Filter  imports and exports and add net exports to demand
    im_ex_DE = dispatch_results[[col for col in dispatch_results.columns if "link_" in col]].copy()
    export_links = [col for col in im_ex_DE if "('DE_link_" in col]
    import_links = [col for col in im_ex_DE if "DE_bus_el')" in col]
    im_ex_DE["overall_exports"] = im_ex_DE[export_links].sum(axis=1)
    im_ex_DE["overall_imports"] = -im_ex_DE[import_links].sum(axis=1)
    im_ex_DE["net_export"] = im_ex_DE["overall_exports"] + im_ex_DE["overall_imports"]
    
    # Resample (different frequency)
    im_ex_DE.loc["2051-01-01 00:00:00"] = im_ex_DE.iloc[-1]
    im_ex_DE.index = pd.to_datetime(im_ex_DE.index)
    im_ex_DE = im_ex_DE.div(multiplier[freq])
    im_ex_DE = im_ex_DE.resample("H").interpolate("ffill")[:-1]
    
    demand_after_flex += im_ex_DE["net_export"]
    
    # Filter electrical storage results; add storage outflow and subtract storage inflow
    storage_DE = dispatch_results[[col for col in dispatch_results.columns if "DE_storage_el" in col]].copy()
    storage_inflow_cols = [col for col in storage_DE if "('DE_bus_el'," in col]
    storage_outflow_cols = [col for col in storage_DE if ", 'DE_bus_el')" in col]

In [ ]:
storage_inflow

In [ ]:
demand_after_flex = original_demand[dr_scenario].copy()

In [ ]:
dispatch_results[[col for col in dispatch_results.columns if "storage" in col]]